In [1]:
import numpy as np
import pandas as pd
import pyddm
import os
import models
import utils

In [2]:
def get_model_measures(model, condition):
    sol = model.solve(condition)
    mean_rt_go = np.sum(sol.pdf(choice="Go")*model.t_domain())*model.dt / sol.prob(choice="Go")
    mean_rt_stay = np.sum(sol.pdf(choice="Stay")*model.t_domain())*model.dt / sol.prob(choice="Stay")
    
    return condition["tta_0"], condition["d_0"], condition["a_values"], condition["a_duration"],\
           sol.prob(choice="Go"), mean_rt_go, mean_rt_stay

def get_model_rt_distr(model, condition, kind="cdf"):
    sol = model.solve(condition)
    
    return pd.DataFrame({"tta_0": condition["tta_0"],
                         "d_0": condition["d_0"],
                         "a_values": str(condition["a_values"]),
                         "a_duration": condition["a_duration"],
                         "t": model.t_domain(),
                         "rt_go_distr": (sol.cdf(choice="Go") if kind=="cdf" else sol.pdf(choice="Go"))/sol.prob(choice="Go"),
                         "rt_stay_distr": (sol.cdf(choice="Stay") if kind=="cdf" else sol.pdf(choice="Stay"))/sol.prob(choice="Stay")})

def initialize_model(model_no, param_set, state_interpolators, T_dur):
    overlay = models.OverlayNonDecisionGaussian(ndt_location=param_set.ndt_location, ndt_scale=param_set.ndt_scale)   
    
    # Initialize drift - it's always an instance of DriftAccelerationDependent, but for models 1 to 4 beta_a is set to 0 during fitting 
    if model_no in [1, 2, 3, 4]:
        drift = models.DriftAccelerationDependent(alpha=param_set.alpha, beta_d=param_set.beta_d, beta_a=0, theta=param_set.theta, state_interpolators=state_interpolators)
    elif model_no in [5, 6, 7, 8]:
        drift = models.DriftAccelerationDependent(alpha=param_set.alpha, beta_d=param_set.beta_d, beta_a=param_set.beta_a, theta=param_set.theta, state_interpolators=state_interpolators)
    
    # Initialize bound - it's constant for models 1, 2, 5, 6 and collapsing with TTA for models 3, 4, 7, 8
    if model_no in [1, 2, 5, 6]:
        bound = pyddm.BoundConstant(B=param_set.B)
    elif model_no in [3, 4, 7, 8]:
        bound = models.BoundCollapsingTta(b_0=param_set.b_0, k=param_set.k, tta_crit=param_set.tta_crit, state_interpolators=state_interpolators)
    
    # Initialize IC - it's fixed at 0 for models 1, 3, 5, 7 and fittable for models 2, 4, 6, 8
    if model_no in [1, 3, 5, 7]:
        IC = pyddm.ICPointRatio(x0=0)
    elif model_no in [2, 4, 6, 8]:
        IC = pyddm.ICPointRatio(x0=param_set.x0)
        
    model = pyddm.Model(name="Model %i" % model_no, drift=drift, bound=bound, overlay=overlay, IC=IC,
                      noise=pyddm.NoiseConstant(noise=1), T_dur=T_dur, choice_names=("Go", "Stay"))        
    return model

def simulate_model(model_no, param_set, conditions, ret="measures", T_dur=4):
    """
    Set ret to "measures" or "rt_cdf" or "rt_pdf" for saving p_turn and mean RT or RT CDF or RT PDF
    """
    state_interpolators = models.get_state_interpolators(conditions=conditions)
    model = initialize_model(model_no, param_set, state_interpolators, T_dur)
    if ret=="measures":
        sim_result = pd.DataFrame([get_model_measures(model, condition) for condition in conditions],
                                  columns=["tta_0", "d_0", "a_values", "a_duration", "is_go_decision", "RT_go", "RT_stay"])
    elif ((ret=="rt_cdf") | (ret=="rt_pdf")):
        sim_result = pd.concat([get_model_rt_distr(model, condition, kind=ret[-3:]) for condition in conditions])
    else:
        raise Exception("ret should be either 'measures' or 'rt_cdf' or 'rt_pdf'")
    sim_result["subj_id"] = param_set.subj_id
    return sim_result

In [9]:
def save_sim_results(loss, model_no, parameters=None, conditions=None, ret="measures", prefix="", T_dur=4):
    file_name="subj_all_parameters_fitted.csv"
    path = os.path.join("modeling/fit_results_%s" % (loss), "model_%i" % (model_no))

    if parameters is None:
        parameters = pd.read_csv(os.path.join(path, file_name))

    sim_results = [simulate_model(model_no, param_set, conditions, ret=ret, T_dur=T_dur)
                   for idx, param_set in parameters.iterrows()]

    sim_results = pd.concat(sim_results)
    sim_results.to_csv(os.path.join(path, (prefix + file_name).replace("parameters_fitted", "sim_" + ret)), index=False)

# Saving model-predicted measures

In [18]:
loss = "bic"
for model_no in range(1, 9):
    save_sim_results(loss=loss, model_no=model_no, conditions=models.get_conditions(), ret="measures", T_dur=4)

In [13]:
save_sim_results(loss=loss, model_no=2, conditions=models.get_conditions(), ret="rt_pdf", T_dur=4)

In [13]:
save_sim_results(loss="vincent", model_no=2, conditions=models.get_conditions(), ret="measures")

# Predicting the effect of other nudges

In [14]:
a_durations = np.linspace(0.1, 2.0, 11)
a_magnitudes = np.linspace(0.0, 5.0, 11)

conditions = [{"tta_0": 6, "d_0": 90, "a_values": (0.0, -a_magnitude, a_magnitude, 0.0), "a_duration": a_duration}
              for a_duration in a_durations
              for a_magnitude in a_magnitudes]

In [15]:
loss = "bic"
model_no = 2
save_sim_results(loss=loss, model_no=model_no, conditions=conditions, ret="measures", prefix="prediction_", T_dur=4)